In [1]:
!pip install -q huggingface_hub[hf_xet]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 56.1 MB/s eta 0:00:00


In [2]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HUGGINGFACE_TOKEN")

login(token = hf_token)

In [3]:
from datasets import load_dataset

daa_file_path='/kaggle/input/building-the-fine-tuning-dataset/train.csv'

train_ds=load_dataset('csv', data_files=daa_file_path)['train']
train_ds

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['anchor', 'positive'],
    num_rows: 4984
})

In [4]:
train_ds[0]

{'anchor': '<think>\nLet’s think through this step by step\nApril: 48 clips\nMay: 48 / 2 = 24 clips\nTotal: 48 + 24 = 72 clips\n</think>\n\\boxed{72}',
 'positive': '<think>\nLet’s think through this step by step\nApril: 48 clips\nMay: 48 / 2 = 24 clips\nTotal = 48 + 24 = 72 clips\n</think>\n\\boxed{72}'}

# Pre-processing dataset

We will split dataset into training, testing and evaluation.

In [5]:
train_test_split=train_ds.train_test_split(test_size=0.1, seed=42)
train_test_split

DatasetDict({
    train: Dataset({
        features: ['anchor', 'positive'],
        num_rows: 4485
    })
    test: Dataset({
        features: ['anchor', 'positive'],
        num_rows: 499
    })
})

In [6]:
train_eval_split=train_test_split['train'].train_test_split(test_size=0.1, seed=42)
train_eval_split

DatasetDict({
    train: Dataset({
        features: ['anchor', 'positive'],
        num_rows: 4036
    })
    test: Dataset({
        features: ['anchor', 'positive'],
        num_rows: 449
    })
})

In [7]:
from datasets import DatasetDict

findal_ds=DatasetDict({
    'train': train_eval_split['train'],
    'test': train_eval_split['test'],
    'eval': train_test_split['test']
})
findal_ds

DatasetDict({
    train: Dataset({
        features: ['anchor', 'positive'],
        num_rows: 4036
    })
    test: Dataset({
        features: ['anchor', 'positive'],
        num_rows: 449
    })
    eval: Dataset({
        features: ['anchor', 'positive'],
        num_rows: 499
    })
})

In [8]:
MODEL_NAME='all-MiniLM-L6-v2'
OUTPUT_DIR='encoder-L6-V2/training'
MODEL_DIR='aisuko/encoder-L6-V2'

# Load the Model

In [9]:
# https://www.sbert.net/docs/package_reference/sentence_transformer/losses.html#multiplenegativesrankingloss
import torch
from sentence_transformers import SentenceTransformer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model=SentenceTransformer(MODEL_NAME).to(device)
model

2025-07-30 12:32:40.004239: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753878760.255376      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753878760.327647      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [10]:
from sentence_transformers.losses import MultipleNegativesRankingLoss

loss=MultipleNegativesRankingLoss(model)

In [11]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, SimilarityFunction

eval_dataset = load_dataset("sentence-transformers/stsb", split="validation")

dev_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=eval_dataset["sentence1"],
    sentences2=eval_dataset["sentence2"],
    scores=eval_dataset["score"],
    main_similarity=SimilarityFunction.COSINE,
    name="sts-dev",
)

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/471k [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/142k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/108k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5749 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1379 [00:00<?, ? examples/s]

In [12]:
from sentence_transformers import SentenceTransformerTrainingArguments
from sentence_transformers.training_args import BatchSamplers
from sentence_transformers import SentenceTransformerTrainer
from transformers import EarlyStoppingCallback


train_args=SentenceTransformerTrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    warmup_ratio=0.1,
    fp16=True,
    bf16=False,
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # losses that use "in-batch negatives" benefit from no duplicates
    eval_strategy='steps',
    eval_steps=100,
    save_strategy='steps',
    save_steps=100,
    save_total_limit=2,
    logging_steps=100,
    report_to='tensorboard',
    run_name='encoder-L6-V2'
)

# evaluate the base model
dev_evaluator(model)

trainer=SentenceTransformerTrainer(
    model=model,
    args=train_args,
    train_dataset=findal_ds['train'],
    eval_dataset=findal_ds['test'],
    loss=loss,
    evaluator=dev_evaluator,
    
)

# early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=3)

# trainer.train(callbacks=[early_stopping_callback])

trainer.train()

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss,Validation Loss,Sts-dev Pearson Cosine,Sts-dev Spearman Cosine
100,0.020400,0.001827,0.876856,0.874318
200,0.004600,0.002021,0.877172,0.875197
300,0.002100,0.001210,0.877318,0.875236
400,0.000800,0.001560,0.877853,0.875684
500,0.002700,0.001355,0.877614,0.875489


TrainOutput(global_step=506, training_loss=0.006051715552206832, metrics={'train_runtime': 1432.604, 'train_samples_per_second': 5.634, 'train_steps_per_second': 0.353, 'total_flos': 0.0, 'train_loss': 0.006051715552206832, 'epoch': 2.0})

In [13]:
# evaluate the trained model on the test set
test_dataset = load_dataset("sentence-transformers/stsb", split="test")

test_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=eval_dataset["sentence1"],
    sentences2=eval_dataset["sentence2"],
    scores=eval_dataset["score"],
    main_similarity=SimilarityFunction.COSINE,
    name="sts-test",
)

test_evaluator(model)

{'sts-test_pearson_cosine': 0.877613066248591,
 'sts-test_spearman_cosine': 0.875487017524482}

In [14]:
model.save_pretrained(MODEL_DIR)

In [15]:
import kagglehub

VARIATION_SLUG='v0.1.4'

handler = f'aisuko/encoder-L6-V2/transformers/{VARIATION_SLUG}'

kagglehub.model_upload(
    handle=handler,
    local_model_dir=MODEL_DIR,
    license_name='MIT',
    version_notes='FT with reasoning paths'
)

Uploading Model https://www.kaggle.com/models/aisuko/encoder-L6-V2/transformers/v0.1.4 ...
Starting upload for file aisuko/encoder-L6-V2/sentence_bert_config.json


Uploading: 100%|██████████| 53.0/53.0 [00:01<00:00, 42.2B/s]

Upload successful: aisuko/encoder-L6-V2/sentence_bert_config.json (53B)
Starting upload for file aisuko/encoder-L6-V2/config.json


Uploading: 100%|██████████| 617/617 [00:00<00:00, 719B/s]  

Upload successful: aisuko/encoder-L6-V2/config.json (617B)
Starting upload for file aisuko/encoder-L6-V2/tokenizer_config.json


...

Uploading: 100%|██████████| 695/695 [00:00<00:00, 836B/s]  

Upload successful: aisuko/encoder-L6-V2/special_tokens_map.json (695B)
Starting upload for file aisuko/encoder-L6-V2/1_Pooling/config.json


Uploading: 100%|██████████| 296/296 [00:00<00:00, 338B/s]  

Upload successful: aisuko/encoder-L6-V2/1_Pooling/config.json (296B)


Your model instance version has been created.
Files are being processed...
See at: https://www.kaggle.com/models/aisuko/encoder-L6-V2/transformers/v0.1.4


In [16]:
trainer.push_to_hub('aisuko/encoder-L6-V2')

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1753878821.504bbaf3a637.13.0:   0%|          | 0.00/7.73k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.56k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/aisuko/training/commit/66c3c7b352b9f71ac32551693bc1bc4108135a8f', commit_message='aisuko/encoder-L6-V2', commit_description='', oid='66c3c7b352b9f71ac32551693bc1bc4108135a8f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/aisuko/training', endpoint='https://huggingface.co', repo_type='model', repo_id='aisuko/training'), pr_revision=None, pr_num=None)